In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
import os
import numpy as np
from PIL import Image
import torchvision.models as models
import copy
from torchvision.utils import save_image
import PIL
from models import Generator, Discriminator
import training_helper as th
from torch.autograd import Variable
import training_helper as th

In [2]:
#preperations
trainset = th.imgData(root='hw3_data/p1_npy')
trainset_loader = DataLoader(trainset, batch_size=2048, shuffle=True, num_workers=0)

device = th.getCudaDevice(cudaNum = 1, torchSeed = 123)

Device used: cuda:1


In [ ]:
# training
G = Generator().to(device)
D = Discriminator().to(device)
criterion = nn.BCELoss()

# setup optimizer
optimizerG = optim.Adam(G.parameters(), lr=0.0004, betas=(0.5, 0.999))
optimizerD = optim.Adam(D.parameters(), lr=0.0004, betas=(0.5, 0.999))

#th.loadModel('p2_latest_G.pth',G,optimizerG)
#th.loadModel('p2_latest_D.pth',D,optimizerD)

D_loss_list = np.load("hw3_data/p2_plot_npy/D_loss_list.npy").tolist()
G_loss_list = np.load("hw3_data/p2_plot_npy/G_loss_list.npy").tolist()
D_fake_acc_list = np.load("hw3_data/p2_plot_npy/D_fake_acc_list.npy").tolist()
D_real_acc_list = np.load("hw3_data/p2_plot_npy/D_real_acc_list.npy").tolist()

D_loss_list = []
G_loss_list = []
D_fake_acc_list = []
D_real_acc_list = []

fixedNoise = Variable(torch.randn(32, 100, 1, 1)).to(device)

while len(D_loss_list) < 100:
    print("Epoch:", len(D_loss_list))
    epoch_D_loss, epoch_G_loss, D_fake_acc, D_real_acc  = 0.0, 0.0, 0.0, 0.0
    
    if len(D_loss_list) % 25 == 0:
        optimizerG.param_groups[0]['lr'] /= 2
        optimizerD.param_groups[0]['lr'] /= 2
        print("lr changed")
        
    for batch_idx, data in enumerate(trainset_loader):
        D.zero_grad()
        
        #### train with real image -> ground truth = real label
        real_image = Variable(data).to(device)
        real_label = Variable(torch.ones(len(data))).to(device)
        output = D(real_image)
        D_real_loss = criterion(output, real_label)
        D_real_acc += np.sum(((output > 0.5).cpu().data.numpy() == real_label.cpu().data.numpy()))
        
        #### train with fake image -> ground truth = fake label
        noise = Variable(torch.randn((len(data),100,1,1))).to(device)
        fake_image = G(noise)
        fake_label = Variable(torch.zeros(len(data))).to(device)
        output = D(fake_image.detach())
        D_fake_loss = criterion(output, fake_label)
        D_fake_acc += np.sum(((output > 0.5).cpu().data.numpy() == fake_label.cpu().data.numpy()))
        
        # update D
        D_train_loss = D_real_loss + D_fake_loss
        epoch_D_loss += float(D_train_loss.data)
        D_train_loss.backward()
        optimizerD.step()
        
        #### train Generator
        gIte = 1
        for i in range(gIte):
            G.zero_grad()
            # generate fake image
            noise = Variable(torch.randn(len(data),100,1,1)).to(device)
            fake_image = G(noise)
            fake_label_for_G = Variable(torch.ones(len(data))).to(device)
            output = D(fake_image)
            G_loss = criterion(output, fake_label_for_G)
            epoch_G_loss += float(G_loss.data)/gIte
            G_loss.backward()
            optimizerG.step()
    
    print("training D Loss:",epoch_D_loss/len(trainset))
    print("training G Loss:", epoch_G_loss/len(trainset))
    D_loss_list.append(epoch_D_loss/len(trainset))
    G_loss_list.append(epoch_G_loss/len(trainset))
    
    print("D_real_acc:", D_real_acc/len(trainset))
    print("D_fake_acc:", D_fake_acc/len(trainset))
    print('')
    
    D_real_acc_list.append(D_real_acc/len(trainset))
    D_fake_acc_list.append(D_fake_acc/len(trainset))
    
    
    th.saveModel('p2_latest_G.pth',G,optimizerG)
    th.saveModel('p2_latest_D.pth',D,optimizerD)
    
    G.eval()
    generatedImages = G(fixedNoise)
    G.train()
    
    torchvision.utils.save_image(generatedImages.cpu().data, 'hw3_data/p2_generator_images/ep-'+str(len(D_loss_list)-1)+'.jpg', nrow=8)
    np.save("hw3_data/p2_plot_npy/D_loss_list.npy", D_loss_list)
    np.save("hw3_data/p2_plot_npy/G_loss_list.npy", G_loss_list)
    np.save("hw3_data/p2_plot_npy/D_real_acc_list.npy", D_real_acc_list)
    np.save("hw3_data/p2_plot_npy/D_fake_acc_list.npy", D_fake_acc_list)
    

Epoch: 0
lr changed
training D Loss: 0.00019683934738859535
training G Loss: 0.003211895942687988
D_real_acc: 0.924625
D_fake_acc: 0.933925

model saved to p2_latest_G.pth
model saved to p2_latest_D.pth
Epoch: 1
training D Loss: 0.0003214646354143042
training G Loss: 0.005717286694049835
D_real_acc: 0.908325
D_fake_acc: 0.84935

model saved to p2_latest_G.pth
model saved to p2_latest_D.pth
Epoch: 2
training D Loss: 0.00032633765414357183
training G Loss: 0.003469477987289429
D_real_acc: 0.9089
D_fake_acc: 0.85985

model saved to p2_latest_G.pth
model saved to p2_latest_D.pth
Epoch: 3
training D Loss: 0.00037180236987769603
training G Loss: 0.002066770645976067
D_real_acc: 0.8101
D_fake_acc: 0.806125

model saved to p2_latest_G.pth
model saved to p2_latest_D.pth
Epoch: 4
training D Loss: 0.0004962743058800698
training G Loss: 0.0016637030623853208
D_real_acc: 0.8003
D_fake_acc: 0.82455

model saved to p2_latest_G.pth
model saved to p2_latest_D.pth
Epoch: 5
training D Loss: 0.00047029323